In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import core
import keras
import core.keras_models as keras_models

PLOTS_DIR = f"plots/regression_transformer/"
MODEL_DIR = f"models/regression_transformer/"
CONFIG_PATH = "../CONDOR/training/load_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], max_events=4_000_000, event_numbers="even"
)

X_train, y_train = DataProcessor.get_data()
del DataProcessor  # Free memory

2026-02-04 12:53:20.899884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770206000.923079 3475437 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770206000.930751 3475437 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770206000.949336 3475437 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770206000.949356 3475437 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770206000.949359 3475437 computation_placer.cc:177] computation placer alr

In [2]:
Transformer = keras_models.FeatureConcatAssigner(data_config, name="Transformer")

In [3]:
Transformer.build_model(
    hidden_dim=64,
    num_layers=6,
    dropout_rate=0.1,
    use_global_event_features=True,
)

I0000 00:00:1770206106.383141 3475437 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6663 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Adding normalization for global event features
Building model without regression output.


In [4]:
Transformer.adapt_normalization_layers(X_train)
Transformer.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy(name="assignment_accuracy")],
    },
    loss_weights={"assignment": 1.0}
)

Adapting normalization layer:  hlf_input_normalization
Processing input tensor:  jet_inputs
Processing input tensor:  lep_inputs
Processing input tensor:  met_inputs
Processing input tensor:  global_event_inputs


I0000 00:00:1770206108.362846 3477529 service.cc:152] XLA service 0x7f365c003e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1770206108.362870 3477529 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2026-02-04 12:55:08.385490: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1770206108.440233 3477529 cuda_dnn.cc:529] Loaded cuDNN version 91500
I0000 00:00:1770206108.731305 3477529 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Adapted normalization layer:  hlf_input_normalization
Adapting normalization layer:  global_event_input_normalization
Processing input tensor:  jet_inputs
Processing input tensor:  lep_inputs
Processing input tensor:  met_inputs
Processing input tensor:  global_event_inputs
Adapted normalization layer:  global_event_input_normalization
Adapting normalization layer:  jet_input_normalization
Processing input tensor:  jet_inputs
Processing input tensor:  lep_inputs
Processing input tensor:  met_inputs
Processing input tensor:  global_event_inputs
Adapted normalization layer:  jet_input_normalization
Adapting normalization layer:  met_input_normalization
Processing input tensor:  jet_inputs
Processing input tensor:  lep_inputs
Processing input tensor:  met_inputs
Processing input tensor:  global_event_inputs
Adapted normalization layer:  met_input_normalization
Adapting normalization layer:  lep_input_normalization
Processing input tensor:  jet_inputs
Processing input tensor:  lep_inputs
P

In [5]:
Transformer.model.summary()

Model: "FeatureConcatTransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 5)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6)         │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ compute_high_level… │ (None, 6, 2, 2)   │          0 │ jet_input_transf… │
│ (ComputeHighLevelF… │                   │            │ jet_mask[0][0],   │
│                     │                   │            │ lep_input_transf… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_event_inputs │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hlf_input_normaliz… │ (None, 6, 2, 2)   │          9 │ compute_high_lev… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_event_input… │ (None, 3)         │          7 │ global_event_inp… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 5)      │         11 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 6, 4)      │          0 │ hlf_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3)         │          0 │ global_event_inp… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 9)      │          0 │ jet_input_normal… │
│ (Concatenate)       │                   │            │ reshape[0][0]   

 Total params: 207,314 (809.84 KB)

 Trainable params: 207,273 (809.66 KB)

 Non-trainable params: 41 (184.00 B)

In [6]:
Transformer.train_model(
    epochs=10,
    X_train=X_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
    validation_split=0.1,
)

Epoch 1/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 96s 64ms/step - assignment_accuracy: 0.4451 - loss: 0.1253 - val_assignment_accuracy: 0.6880 - val_loss: 0.0781 - learning_rate: 1.0000e-04
Epoch 2/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - assignment_accuracy: 0.6599 - loss: 0.0853 - val_assignment_accuracy: 0.7236 - val_loss: 0.0718 - learning_rate: 1.0000e-04
Epoch 3/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - assignment_accuracy: 0.6902 - loss: 0.0782 - val_assignment_accuracy: 0.7431 - val_loss: 0.0690 - learning_rate: 1.0000e-04
Epoch 4/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - assignment_accuracy: 0.7103 - loss: 0.0744 - val_assignment_accuracy: 0.7573 - val_loss: 0.0672 - learning_rate: 1.0000e-04
Epoch 5/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - assignment_accuracy: 0.7232 - loss: 0.0723 - val_assignment_accuracy: 0.7632 - val_loss: 0.0659 - learning_rate: 1.0000e-04
Epoch 6/10
878/878 ━━━━━━━━━━━━━━━━━━━━ 27s 30ms/step - assignment_accuracy: 0.7354 - loss: 0.0703 -

KeyboardInterrupt: 

In [ ]:
Transformer.save_model(MODEL_DIR + "odd_model.keras")